In [5]:
import psycopg2
import pandas as pd
import configparser
import matplotlib.pyplot as plt

# Read from the Config file
config = configparser.ConfigParser()
config.read_file(open(r'../db_ops.cfg'))

con_str = config.get('con_str', 'PG_AIRBASES') 
engine = sqlalchemy.create_engine(con_str)

try:
    connection = engine.connect()
    print ("Opened Connection")
except (Exception, sqlalchemy.exc.SQLAlchemyError) as error:
    print("Error while connecting to PostgreSQL database:", error)


except (Exception, psycopg2.Error) as error:
    print("Error while executing the query:", error)
finally:
    # Close the connection
    if connection:
        connection.close()
        print("Connection closed")


Opened Connection
Connection closed
